# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 4 2022 10:30AM,247827,19,N17792,"NAPP Technologies, LLC",Released
1,Oct 4 2022 10:29AM,247826,19,8608812,"Emersa Waterbox, LLC",Released
2,Oct 4 2022 10:23AM,247825,10,8609364,Eywa Pharma Inc.,Released
3,Oct 4 2022 10:16AM,247824,10,Enova-10754,Emerginnova,Released
4,Oct 4 2022 10:16AM,247824,10,Enova-10755,Emerginnova,Released
5,Oct 4 2022 10:15AM,247822,10,snap8593610,"Snap Medical Industries, LLC",Released
6,Oct 4 2022 10:15AM,247822,10,8603749,"Snap Medical Industries, LLC",Released
7,Oct 4 2022 10:15AM,247822,10,8606205,"Snap Medical Industries, LLC",Released
8,Oct 4 2022 10:15AM,247822,10,8606206,"Snap Medical Industries, LLC",Released
9,Oct 4 2022 10:15AM,247822,10,8606207,"Snap Medical Industries, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,247822,Released,6
25,247824,Released,2
26,247825,Released,1
27,247826,Released,1
28,247827,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247822,NaN,NaN,6.0
247824,NaN,NaN,2.0
247825,NaN,NaN,1.0
247826,NaN,NaN,1.0
247827,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247719,0.0,0.0,1.0
247726,0.0,0.0,1.0
247736,0.0,1.0,1.0
247757,0.0,0.0,3.0
247764,0.0,0.0,12.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247719,0,0,1
247726,0,0,1
247736,0,1,1
247757,0,0,3
247764,0,0,12


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247719,0,0,1
1,247726,0,0,1
2,247736,0,1,1
3,247757,0,0,3
4,247764,0,0,12


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247719,,,1
1,247726,,,1
2,247736,,1,1
3,247757,,,3
4,247764,,,12


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 4 2022 10:30AM,247827,19,"NAPP Technologies, LLC"
1,Oct 4 2022 10:29AM,247826,19,"Emersa Waterbox, LLC"
2,Oct 4 2022 10:23AM,247825,10,Eywa Pharma Inc.
3,Oct 4 2022 10:16AM,247824,10,Emerginnova
5,Oct 4 2022 10:15AM,247822,10,"Snap Medical Industries, LLC"
11,Oct 4 2022 10:08AM,247821,10,"Snap Medical Industries, LLC"
12,Oct 4 2022 9:56AM,247820,10,Eywa Pharma Inc.
32,Oct 4 2022 9:53AM,247815,10,Bio-PRF
46,Oct 4 2022 9:42AM,247819,10,"Virtue Rx, LLC"
47,Oct 4 2022 9:39AM,247816,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 4 2022 10:30AM,247827,19,"NAPP Technologies, LLC",,,1
1,Oct 4 2022 10:29AM,247826,19,"Emersa Waterbox, LLC",,,1
2,Oct 4 2022 10:23AM,247825,10,Eywa Pharma Inc.,,,1
3,Oct 4 2022 10:16AM,247824,10,Emerginnova,,,2
4,Oct 4 2022 10:15AM,247822,10,"Snap Medical Industries, LLC",,,6
5,Oct 4 2022 10:08AM,247821,10,"Snap Medical Industries, LLC",,,1
6,Oct 4 2022 9:56AM,247820,10,Eywa Pharma Inc.,,,20
7,Oct 4 2022 9:53AM,247815,10,Bio-PRF,,,14
8,Oct 4 2022 9:42AM,247819,10,"Virtue Rx, LLC",,,1
9,Oct 4 2022 9:39AM,247816,10,ISDIN Corporation,,,16


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 4 2022 10:30AM,247827,19,"NAPP Technologies, LLC",1,,
1,Oct 4 2022 10:29AM,247826,19,"Emersa Waterbox, LLC",1,,
2,Oct 4 2022 10:23AM,247825,10,Eywa Pharma Inc.,1,,
3,Oct 4 2022 10:16AM,247824,10,Emerginnova,2,,
4,Oct 4 2022 10:15AM,247822,10,"Snap Medical Industries, LLC",6,,
5,Oct 4 2022 10:08AM,247821,10,"Snap Medical Industries, LLC",1,,
6,Oct 4 2022 9:56AM,247820,10,Eywa Pharma Inc.,20,,
7,Oct 4 2022 9:53AM,247815,10,Bio-PRF,14,,
8,Oct 4 2022 9:42AM,247819,10,"Virtue Rx, LLC",1,,
9,Oct 4 2022 9:39AM,247816,10,ISDIN Corporation,16,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 4 2022 10:30AM,247827,19,"NAPP Technologies, LLC",1,,
1,Oct 4 2022 10:29AM,247826,19,"Emersa Waterbox, LLC",1,,
2,Oct 4 2022 10:23AM,247825,10,Eywa Pharma Inc.,1,,
3,Oct 4 2022 10:16AM,247824,10,Emerginnova,2,,
4,Oct 4 2022 10:15AM,247822,10,"Snap Medical Industries, LLC",6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 4 2022 10:30AM,247827,19,"NAPP Technologies, LLC",1,NaN,NaN
1,Oct 4 2022 10:29AM,247826,19,"Emersa Waterbox, LLC",1,NaN,NaN
2,Oct 4 2022 10:23AM,247825,10,Eywa Pharma Inc.,1,NaN,NaN
3,Oct 4 2022 10:16AM,247824,10,Emerginnova,2,NaN,NaN
4,Oct 4 2022 10:15AM,247822,10,"Snap Medical Industries, LLC",6,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 4 2022 10:30AM,247827,19,"NAPP Technologies, LLC",1,0.0,0.0
1,Oct 4 2022 10:29AM,247826,19,"Emersa Waterbox, LLC",1,0.0,0.0
2,Oct 4 2022 10:23AM,247825,10,Eywa Pharma Inc.,1,0.0,0.0
3,Oct 4 2022 10:16AM,247824,10,Emerginnova,2,0.0,0.0
4,Oct 4 2022 10:15AM,247822,10,"Snap Medical Industries, LLC",6,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4955884,197,31.0,4.0
16,247796,7,1.0,0.0
19,743433,3,0.0,0.0
20,247719,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4955884,197,31.0,4.0
1,16,247796,7,1.0,0.0
2,19,743433,3,0.0,0.0
3,20,247719,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,197,31.0,4.0
1,16,7,1.0,0.0
2,19,3,0.0,0.0
3,20,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,197.0
1,16,Released,7.0
2,19,Released,3.0
3,20,Released,1.0
4,10,Executing,31.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20
Status,,,,
Completed,4.0,0.0,0.0,0.0
Executing,31.0,1.0,0.0,0.0
Released,197.0,7.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20
0,Completed,4.0,0.0,0.0,0.0
1,Executing,31.0,1.0,0.0,0.0
2,Released,197.0,7.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20
0,Completed,4.0,0.0,0.0,0.0
1,Executing,31.0,1.0,0.0,0.0
2,Released,197.0,7.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()